# Compression de Huffman

Idée de codage de Huffman: utiliser moins de bits pour les lettres qui apparaissent souvent.

**Idée de l'Algorithme:**  
* Prétraitement: On construit un arbre binaire T qu'on appelle arbre de Huffman dont les feuilles sont les lettres. 
* Compression: On code chaque lettre c par une suite de 0 et 1 correpondant au chemin de la racine à la feuille contenant c dans T.  
* Décompression: On décode une suite de 0 et 1 en parcourant le chemin correspondant dans T.

In [28]:
let get_frequence text = (*trouve la fréquence des caractères pour créer l'arbre*)
    let freq = Array.make 256 0 in
    for i = 0 to String.length text -1 do
        freq.(Char.code text.[i]) <- freq.(Char.code text.[i]) + 1
    done;
    freq;;

val get_frequence : string -> int array = <fun>

**Construction de l'arbre de Huffman**

In [4]:
type 'a tree = F of 'a | N of 'a tree * 'a tree

type 'a tree = F of 'a | N of 'a tree * 'a tree

q <- file de priorité contenant F(c) (un arbre) avec la priorité f(c), pour chaque lettre c  
Tant que q contient >= 2 éléments:  
&ensp;&thinsp;&ensp;&thinsp;&ensp;&thinsp;Extraire de q les 2 arbres t1 et t2 de priorité min f1 et f2  
&ensp;&thinsp;&ensp;&thinsp;&ensp;&thinsp;Ajouter à q l'arbre N(t1, t2) avec la priorité f1 + f2  
L'arbre restant à la fin est l'abre de Huffman

In [21]:
let make_huffman_tree freq = 
    let q = Queue.empty () in
    let n = ref 0 in
    for i = 0 to 255 do
        if freq.(i) > 0 then(
        incr n;
        Queue.add q (freq.(i), F(Char.chr i)))
    done;
    for _ = 1 to !n-2 do
        let f1, t1 = Queue.take_min q in
        let f2, t2 = Queue.take_min q in
        Queue.add q (f1+f2, N(t1, t2))
    done;
    snd(Queue.take_min q);;

Line 2, characters 12-23:
Error: Unbound value Queue.empty


In [22]:
let make_table t = 
    let code = Array.make 256 [] in
    let rec aux path = function
        |F(c) -> code.(Char.code c) <- List.rev path
        |N(g, d) -> aux (0::path) g; aux (1::path) d in
    aux [] t;
    code

val make_table : char tree -> int list array = <fun>

In [23]:
let compress_huffman text = 
    let freq = get_frequence text in
    let t = make_huffman freq in
    let table = make_table t in
    let rec aux i =
        if i = String.length text then []
        else table.(Char.code text.[i]) @ aux (i+1) in
    aux 0;;

Line 3, characters 12-24:
Error: Unbound value make_huffman


In [27]:
let rec decode_huffman t code = 
    let rec read_char t l = match t,l with
        |F(c), _ -> c, l
        |N(g, d), 0::q -> read_char g q
        |N(g, d), 1::q -> read_char d q
        |_ -> failwith "codage incorrect" in
    if code = [] then ""
    else let c, l = read_char t code in
    (String.make 1 c)^(decode_huffman t l)

val decode_huffman : char tree -> int list -> string = <fun>